In [3]:
import boto3
import json
import base64
import io
import uuid
from PIL import Image

# Set up the Amazon Bedrock client
bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1"
)

# Define the model ID
model_id = "amazon.nova-canvas-v1:0"

# Prepare the input prompt
prompt = "create me an image of a group of 6 girls sitting together in a garden and enjoying picnic"

# Create the request payload
body = json.dumps({
        "taskType": "TEXT_IMAGE",
        "textToImageParams": {
            "text": prompt
        },
        "imageGenerationConfig": {
            "numberOfImages": 1,
            "height": 1024,
            "width": 1024,
            "cfgScale": 8.0,
            "seed": 0
        }
    })

accept = "application/json"
content_type = "application/json"

# Invoke the Amazon Bedrock model
response = bedrock_client.invoke_model(
    modelId=model_id,
    body=body,
    accept=accept, 
    contentType=content_type
)

# Process the response
result = json.loads(response["body"].read())

base64_image = result.get("images")[0]
base64_bytes = base64_image.encode('ascii')
image_bytes = base64.b64decode(base64_bytes)

image = Image.open(io.BytesIO(image_bytes))
image.show()

s3 = boto3.client("s3")
id = uuid.uuid4()

bucket_name = "generated-images-blog2-bucket"
image_filename = f"generated-images/{id}.png"

response = s3.put_object(
            Bucket=bucket_name,
            Key=image_filename,
            Body=image_bytes,
            ContentType='image/jpeg'
        )

s3_url = f"https://{bucket_name}.s3.amazonaws.com/{image_filename}"

print(f"The Generated image is available at: {s3_url}")

The Generated image is available at: https://generated-images-blog2-bucket.s3.amazonaws.com/generated-images/8dbffd18-f477-4ec8-9c9c-c0708a0dd5cc.png
